## **Predicting purchase probabilities**

### **Loading the data**

In [270]:
import pandas as pd
import numpy as np
from copy import deepcopy

from tqdm import tqdm

tqdm.pandas()

In [5]:
DATA_PATH = '/Users/leonhagel/Documents/coding/github/mlim-g2/data/'


data_sets = {
    'baskets': DATA_PATH + 'baskets.parquet',
    'coupon_index': DATA_PATH + 'coupon_index.parquet', 
    'coupons': DATA_PATH + 'coupons.parquet'
}

In [281]:
class Helper:
    
    def __init__(self):
        self.data = {}
        self.mappings = {}
        
    def _load(self, filepath):
        import pandas as pd
        file_type = filepath.split('.')[1]
        
        if file_type == 'parquet':
            output = pd.read_parquet(filepath)
        
        return output

    
    def load(self, files:dict):
        for name, filepath in files.items():
            self.data[name] = self._load(filepath)
    
    
    def _dump_data(self, export_path, name):
        self.data[name].to_parquet(f"{export_path}{name}.parquet")
    
    
    def dump(self, export_path, which='all'):
        if which == 'all':
            # data
            provided_data = ['baskets', 'coupon_index', 'coupons']
            data = [data for data in self.data.keys() if data not in provided_data]
            
            # other attributes and objects
            other = []
            
            which = data + other
        
        for name in which:
            if name in self.data.keys():
                self._dump_data(export_path, name)
        
    # data preparation
    def get_merged(self, drop=False):
        similar = [x for x in self.data['coupons'].columns if x in self.data['baskets'].columns] # i.e. ['shopper', 'product', 'week'] 
        self.data['merged'] = self.data['baskets'].merge(self.data['coupons'], how='left', left_on=similar, right_on=similar)
        if drop:
            self.data.pop('baskets')
            self.data.pop('coupons')    

            
    def reduce_data_size(self, df):
        max_integer_values = {127: 'int8', 32767: 'int16', 2147483647: 'int32'}
        for column, dtype in df.dtypes.items():    
            if np.issubdtype(dtype, np.integer):
                # determining the minimum dtype
                max_value = np.max([abs(df[column].min()), df[column].max()])
                max_array = np.array(list(max_integer_values.keys()))
                max_idx = max_array[max_array > max_value][0]
                # converting integers
                df[column] = df[column].astype(max_integer_values[max_idx])
            # converting float
            if np.issubdtype(dtype, np.floating):
                df[column] = df[column].astype('float32')
        return df
    
    
    def reduce_shopper(self, df, shopper_range:tuple=(0,1999)):
        lower = df['shopper'] >= shopper_range[0]
        upper = df['shopper'] <= shopper_range[1]
        return df.loc[lower & upper]
    
    
    def clean(self, df='merged', shopper_range=(0,1999)):
        cond = all([(type(df) != pd.core.frame.DataFrame), (df == 'merged')])
        df = self.data['merged'].copy() if cond else df
        df = purchase.reduce_shopper(df, shopper_range)
    
        df['discount'].fillna(0, inplace=True)
        df['discount'] = df['discount'] / 100
    
        df['price'] = df['price'] / (1 - df['discount'])
        return df
    
    
    # data mappings
    def _init_df_map(self, rows, columns, initial_array=[]):
        rows = range(rows[0], rows[1]+1) if type(rows) == tuple else rows
        columns = range(columns[0], columns[1]+1) if type(columns) == tuple else columns

        rows = {row: deepcopy(initial_array) for row in rows}
        return pd.DataFrame({column: deepcopy(rows) for column in columns})

In [282]:
class Purchase_Probabilities(Helper):
    
    def __init__(self):
        super().__init__()
        
    

In [283]:
purchase = Purchase_Probabilities()
purchase.load(data_sets)

In [284]:
#purchase.get_merged()
purchase.load({'merged': DATA_PATH + 'merged.parquet'})
purchase.data['merged'] = purchase.reduce_data_size(purchase.data['merged'])

### **EDA of raw data**

In [114]:
purchase.data['merged'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68841598 entries, 0 to 68841597
Data columns (total 5 columns):
 #   Column    Dtype  
---  ------    -----  
 0   week      int8   
 1   shopper   int32  
 2   product   int16  
 3   price     int16  
 4   discount  float32
dtypes: float32(1), int16(2), int32(1), int8(1)
memory usage: 1.3 GB


In [115]:
purchase.data['merged'].describe().round(2)

,week,shopper,product,price,discount
count,68841598.00,68841598.00,68841598.00,68841598.00,2708043.00
mean,44.50,50006.96,124.96,584.90,27.29
std,25.98,28881.66,69.84,97.75,9.78
min,0.00,0.00,0.00,234.00,10.00
25%,22.00,24981.00,66.00,506.00,20.00
50%,44.00,50019.00,123.00,579.00,30.00
75%,67.00,75041.00,189.00,654.00,35.00
max,89.00,99999.00,249.00,837.00,40.00


In [116]:
print('missing values:')
purchase.data['merged'].isna().sum()

missing values:


week               0
shopper            0
product            0
price              0
discount    66133555
dtype: int64

In [117]:
purchase.data['coupon_index']

,week,shopper,coupon
0,90,0,0
2000,90,0,1
4000,90,0,2
6000,90,0,3
8000,90,0,4
...,...,...,...
1999,90,1999,0
3999,90,1999,1
5999,90,1999,2
7999,90,1999,3


In [118]:
applied_discounts = purchase.data['merged']['discount'].astype('category').unique()
applied_discounts

/Users/leonhagel/env/mlim-g2/lib/python3.8/site-packages/pandas/io/formats/format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


[NaN, 35.0, 10.0, 25.0, 30.0, 20.0, 15.0, 40.0]
Categories (7, float64): [35.0, 10.0, 25.0, 30.0, 20.0, 15.0, 40.0]

<br>

## **Data preparation**

### **Data cleaning**

**Final function**

In [250]:
# final function
def clean(self, df='merged', shopper_range=(0,1999)):
    cond = all([(type(df) != pd.core.frame.DataFrame), (df == 'merged')])
    df = self.data['merged'].copy() if cond else df
    df = purchase.reduce_shopper(df, shopper_range)
    
    df['discount'].fillna(0, inplace=True)
    df['discount'] = df['discount'] / 100
    
    df['price'] = df['price'] / (1 - df['discount'])
    return df

del clean

<br>

**Copy data**

In [251]:
df = purchase.data['merged'].copy()
df

,week,shopper,product,price,discount
0,0,0,71,629,NaN
1,0,0,91,605,NaN
2,0,0,116,715,NaN
3,0,0,123,483,NaN
4,0,0,157,592,NaN
...,...,...,...,...,...
68841593,89,99999,143,470,NaN
68841594,89,99999,158,566,NaN
68841595,89,99999,186,499,NaN
68841596,89,99999,204,496,NaN


<br>

**Reducing shoppers**

In [252]:
df = purchase.reduce_shopper(df)
df

,week,shopper,product,price,discount
0,0,0,71,629,NaN
1,0,0,91,605,NaN
2,0,0,116,715,NaN
3,0,0,123,483,NaN
4,0,0,157,592,NaN
...,...,...,...,...,...
68091869,89,1999,144,562,NaN
68091870,89,1999,158,566,NaN
68091871,89,1999,192,549,NaN
68091872,89,1999,213,592,NaN


<br>

**Missing values: Discount**

In [253]:
df['discount'].fillna(0, inplace=True)
df['discount'] = df['discount'] / 100

applied_discounts = df['discount'].astype('category').unique()
print(applied_discounts)

df.describe().round(2)

/Users/leonhagel/env/mlim-g2/lib/python3.8/site-packages/pandas/core/series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-253-03e92c5954db>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['discount'] = df['discount'] / 100
/Users/leonhagel/env/mlim-g2/lib/python3.8/site-packages/pandas/io/formats/format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


[0.00, 0.35, 0.10, 0.25, 0.30, 0.20, 0.15, 0.40]
Categories (8, float64): [0.00, 0.35, 0.10, 0.25, 0.30, 0.20, 0.15, 0.40]


,week,shopper,product,price,discount
count,1378720.00,1378720.00,1378720.00,1378720.00,1378720.00
mean,44.48,997.21,125.07,584.34,0.01
std,25.98,576.25,69.70,97.43,0.06
min,0.00,0.00,0.00,234.00,0.00
25%,22.00,499.00,66.00,506.00,0.00
50%,44.00,993.00,123.00,579.00,0.00
75%,67.00,1496.00,189.00,654.00,0.00
max,89.00,1999.00,249.00,837.00,0.40


**Calculating the price without discount**

In [254]:
df['price_discounted'] = df['price'].copy()
df['price'] = df['price'] / (1 - df['discount'])
assert any(df['price'] != df['price_discounted'])
df.describe().round(2)

<ipython-input-254-a2122add2da9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_discounted'] = df['price'].copy()
<ipython-input-254-a2122add2da9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'] / (1 - df['discount'])


,week,shopper,product,price,discount,price_discounted
count,1378720.00,1378720.00,1378720.00,1378720.00,1378720.00,1378720.00
mean,44.48,997.21,125.07,590.73,0.01,584.34
std,25.98,576.25,69.70,92.70,0.06,97.43
min,0.00,0.00,0.00,389.23,0.00,234.00
25%,22.00,499.00,66.00,510.00,0.00,506.00
50%,44.00,993.00,123.00,582.00,0.00,579.00
75%,67.00,1496.00,189.00,657.00,0.00,654.00
max,89.00,1999.00,249.00,837.65,0.40,837.00


In [255]:
purchase.data['purchase'] = purchase.clean()
purchase.data['purchase'].describe().round(2)

,week,shopper,product,price,discount
count,1378720.00,1378720.00,1378720.00,1378720.00,1378720.00
mean,44.48,997.21,125.07,590.73,0.01
std,25.98,576.25,69.70,92.70,0.06
min,0.00,0.00,0.00,389.23,0.00
25%,22.00,499.00,66.00,510.00,0.00
50%,44.00,993.00,123.00,582.00,0.00
75%,67.00,1496.00,189.00,657.00,0.00
max,89.00,1999.00,249.00,837.65,0.40


<br>

### **Mappings**

**Loading clean data**

In [285]:
purchase.data['purchase'] = purchase.clean()
purchase.data['purchase'].describe().round(2)

,week,shopper,product,price,discount
count,1378720.00,1378720.00,1378720.00,1378720.00,1378720.00
mean,44.48,997.21,125.07,590.73,0.01
std,25.98,576.25,69.70,92.70,0.06
min,0.00,0.00,0.00,389.23,0.00
25%,22.00,499.00,66.00,510.00,0.00
50%,44.00,993.00,123.00,582.00,0.00
75%,67.00,1496.00,189.00,657.00,0.00
max,89.00,1999.00,249.00,837.65,0.40


#### **Price map**

In [286]:
df = purchase.data['merged'].copy()
df = purchase.clean(df, shopper_range=(0, df['shopper'].max())).loc[:, ['week', 'product', 'price', 'discount']]
df.describe().round()

,week,product,price,discount
count,68841598.0,68841598.0,68841598.0,68841598.0
mean,44.0,125.0,591.0,0.0
std,26.0,70.0,93.0,0.0
min,0.0,0.0,389.0,0.0
25%,22.0,66.0,510.0,0.0
50%,44.0,123.0,582.0,0.0
75%,67.0,189.0,657.0,0.0
max,89.0,249.0,838.0,0.0


In [297]:
weeks = (df['week'].min(), df['week'].max())
products = (df['product'].min(), df['product'].max())

mapping = purchase._init_df_map(rows=weeks, columns=products, initial_array=[])
mapping


,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
1,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
2,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
3,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
4,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
86,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
87,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
88,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]


In [303]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68841598 entries, 0 to 68841597
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   week      int8   
 1   product   int16  
 2   price     float32
 3   discount  float32
dtypes: float32(2), int16(1), int8(1)
memory usage: 1.2 GB


In [298]:
row = df.iloc[0]
row

week          0.0
product      71.0
price       629.0
discount      0.0
Name: 0, dtype: float32

In [299]:
row_name = 'week'
column_name = 'product'
value_name = 'price'

In [302]:
row = df.iloc[0]
row

week          0.0
product      71.0
price       629.0
discount      0.0
Name: 0, dtype: float32

In [306]:
df.progress_apply(lambda row: mapping.loc[int(row[row_name]), int(row[column_name])].append(row[value_name]), axis = 1)

  6%|▌         | 3968076/68841598 [02:02<33:14, 32521.31it/s]  


KeyboardInterrupt: 

In [ ]:
for week in tqdm(range(91)):
    df.loc[df['week'] < week, ['product', 'price']].groupby(by='product').agg(lambda x:x.value_counts().index[0])

In [313]:

week = 50

agg_fn = lambda x:x.value_counts().index[0]
#agg_fn = np.mean
start = time.time()
print(df.loc[df['week'] < week, ['product', 'price']].groupby(by='product').agg(agg_fn).values)
time.time()-start

[[688.]
 [560.]
 [773.]
 [722.]
 [620.]
 [721.]
 [568.]
 [772.]
 [669.]
 [759.]
 [711.]
 [611.]
 [781.]
 [788.]
 [755.]
 [620.]
 [743.]
 [725.]
 [665.]
 [744.]
 [563.]
 [463.]
 [528.]
 [495.]
 [558.]
 [540.]
 [628.]
 [582.]
 [442.]
 [542.]
 [644.]
 [784.]
 [684.]
 [665.]
 [706.]
 [602.]
 [696.]
 [772.]
 [699.]
 [593.]
 [470.]
 [660.]
 [527.]
 [546.]
 [485.]
 [590.]
 [523.]
 [592.]
 [465.]
 [613.]
 [673.]
 [640.]
 [607.]
 [672.]
 [773.]
 [757.]
 [761.]
 [716.]
 [713.]
 [527.]
 [795.]
 [822.]
 [573.]
 [719.]
 [575.]
 [719.]
 [743.]
 [637.]
 [748.]
 [752.]
 [737.]
 [629.]
 [788.]
 [837.]
 [779.]
 [654.]
 [582.]
 [721.]
 [625.]
 [736.]
 [614.]
 [513.]
 [598.]
 [637.]
 [501.]
 [550.]
 [481.]
 [521.]
 [471.]
 [700.]
 [549.]
 [605.]
 [543.]
 [577.]
 [443.]
 [586.]
 [502.]
 [632.]
 [481.]
 [519.]
 [472.]
 [506.]
 [576.]
 [539.]
 [637.]
 [504.]
 [478.]
 [646.]
 [528.]
 [667.]
 [646.]
 [699.]
 [727.]
 [710.]
 [579.]
 [674.]
 [715.]
 [737.]
 [750.]
 [532.]
 [639.]
 [659.]
 [660.]
 [483.]
 [707.]


18.258657932281494

In [316]:
import scipy

#agg_fn = lambda x:x.value_counts().index[0]
#agg_fn = np.mean
agg_fn = lambda x: scipy.stats.mode(x)[0]

start = time.time()
print(df.loc[df['week'] < week, ['product', 'price']].groupby(by='product').agg(agg_fn).values)
time.time()-start


AttributeError: module 'scipy' has no attribute 'stats'

In [314]:
20*90 /60

30.0